In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/SameerBramhecha/Sentimental_Analyser/main/twitter-suicidal_data.csv')

In [3]:
df.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [4]:
df['intention'].value_counts()

0    5121
1    3998
Name: intention, dtype: int64

In [10]:
!pip install spacy==3.2.4

  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached murmurhash-1.0.7-cp39-cp39-win_amd64.whl (18 kB)
  Using cached cymem-2.0.6-cp39-cp39-win_amd64.whl (36 kB)
  Using cached preshed-3.0.6-cp39-cp39-win_amd64.whl (112 kB)
  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)


In [11]:
!python -m spacy download en_core_web_sm

2022-08-13 11:05:10.788030: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-13 11:05:10.788268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
!pip install beautifulsoup4==4.9.1

In [13]:
!pip install textblob==0.15.3

In [14]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\bramh\appdata\local\temp\pip-req-build-7cdy0uu4
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7692 sha256=49b029e806fc083e3815031e745738e2092432950924271519319aff9fb60291
  Stored in directory: C:\Users\bramh\AppData\Local\Temp\pip-ephem-wheel-cache-99khd4hb\wheels\d1\c3\bb\559fe93e652b51cbc532f17e9693f3b70055f8560cf06c1fb3
Successfully built preprocess-kgptalkie
  Attempting uninstall: preprocess-kgptalkie
    Found existing installation: preprocess-kgptalkie 0.1.3
    Uninstalling preprocess-kgptalkie-0.1.3:
      Successfully uninstalled preprocess-kgptalkie-0.1.3


  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\bramh\AppData\Local\Temp\pip-req-build-7cdy0uu4'


In [15]:
import preprocess_kgptalkie as ps

In [16]:
import re

In [17]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [18]:
df['tweet'] = df['tweet'].apply(lambda x: get_clean(x))

In [19]:
df.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [21]:
tfidf = TfidfVectorizer(max_features = 20000, ngram_range = (1,3), analyzer = 'char')

In [22]:
X = tfidf.fit_transform(df['tweet'])
y = df['intention']

In [24]:
X.shape

(9119, 10404)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [27]:
y_pred = clf.predict(X_test)


In [28]:
print(confusion_matrix(y_test,y_pred))

[[988  72]
 [ 67 697]]


In [29]:
print(accuracy_score(y_test, y_pred))

0.9237938596491229


In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1060
           1       0.91      0.91      0.91       764

    accuracy                           0.92      1824
   macro avg       0.92      0.92      0.92      1824
weighted avg       0.92      0.92      0.92      1824



In [31]:
x = 'i will die alone'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([1], dtype=int64)